In [134]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
# Scientific Computing
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats

import bottleneck as bn

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

from tools.tools import *
from tools.io import *
from tools.methods import *

sns.set(context='talk', style='whitegrid', font_scale=1.3)

In [178]:
def sensitivity_length(N, filt, fu, fv, fzeta):
    '''
    least square method varying length of hexagons
    '''
    M = 3
    llist = np.arange(1, 30, 1).astype(float)
    llist = np.insert(llist, 0, 0.5)
    skew=1
    
    error = []
    error_l = []
    error3 = []
    error_l_ci = []
    for l, L in enumerate(llist):
        # make polygons
        xi, yi = make_n_hexs(L, skew, N, M)

        # find the u,v at the polygon vertices
        ui = fu.ev(xi, yi)
        vi = fu.ev(xi, yi)
        # find zeta at polygon center
        true1 = fzeta.ev(bn.nanmean(xi, axis=1), bn.nanmean(yi, axis=1))
        true2 = np.nanmean( fzeta.ev(xi, yi), axis=1)

        estimate = np.zeros(N)
        for i in range(N):
            estimate[i], _, _ = least_square_method(
                xi[i, :], yi[i, :], ui[i, :], vi[i, :], 'solve')
        
        # compute R2 and confidence interval
        error.append(stats.pearsonr(true2, estimate)[0]**2)
        error_l.append(stats.pearsonr(true1, estimate)[0]**2)
        error_l_ci.append(bootstrap_ci(true1, estimate, N))
        error3.append( np.nanmean( vort_drifters/true1 ) )

    # save in dataframe
    df = pd.DataFrame(index=np.asarray(llist))
    df['error1'] = np.asarray(error)
    df['error2'] = np.asarray(error_l)
    df['error3'] = np.asarray(error3)
    df['ci_low'] = np.asarray(error_l_ci)[:, 0]
    df['ci_high'] = np.asarray(error_l_ci)[:, 1]
    df['filter'] = filt

    return df

def sensitivity_aspect(N, filt, fu, fv, fzeta):
    '''
    least square method varying number of drifter per cluster
    '''
    L = 2
    M = 3
    skewlist = np.arange(1, 30, 1)

    aspect = []
    error = []
    error3 = []
    error_l = []
    error_l_ci = []
    for l, skew in enumerate(skewlist):
        # make polygons
        xi, yi = make_n_hexs(L, skew, N, M)

        # find the u,v at the polygon vertices
        ui = fu.ev(xi, yi)
        vi = fv.ev(xi, yi)
        # find zeta at polygon center
        true1 = fzeta.ev(bn.nanmean(xi, axis=1), bn.nanmean(yi, axis=1))
        true2 = np.nanmean( fzeta.ev(xi, yi), axis=1)

        estimate = np.zeros(N)
        for i in range(N):
            estimate[i], _, _ = least_square_method(
                xi[i, :], yi[i, :], ui[i, :], vi[i, :], 'lstsq')

        # compute aspect ratio, R2 and confidence interval
        aspect.append(calc_aspect(xi[0, :], yi[0, :]))
        error.append(stats.pearsonr(true2, estimate)[0]**2)
        error_l.append(stats.pearsonr(true1, estimate)[0]**2)
        error_l_ci.append(bootstrap_ci(true1, estimate, N))
        error3.append( np.nanmean( vort_drifters/true1 ) )
    
    # save in dataframe
    df = pd.DataFrame(index=np.asarray(aspect))
    df['error1'] = np.asarray(error)
    df['error2'] = np.asarray(error_l)
    df['error3'] = np.asarray(error3)
    df['ci_low'] = np.asarray(error_l_ci)[:, 0]
    df['ci_high'] = np.asarray(error_l_ci)[:, 1]
    df['filter'] = filt

    return df

def sensitivity_number(N, filt, fu, fv, fzeta):
    '''
    least square method varying number of drifter per cluster
    '''
    L = 1
    mlist = np.arange(3, 21)
    skew=1
    
    error = []
    error3 = []
    error_l = []
    error_l_ci = []
    for l, M in enumerate(mlist):
        # make polygons
        xi, yi = make_n_hexs(L, skew, N, M)

        # find the u,v at the polygon vertices
        ui = fu.ev(xi, yi)
        vi = fv.ev(xi, yi)
        # find zeta at polygon center
        true1 = fzeta.ev(bn.nanmean(xi, axis=1), bn.nanmean(yi, axis=1))
        true2 = np.nanmean( fzeta.ev(xi, yi), axis=1)

        # estimate vorticity from the velocities
        estimate = np.zeros(N)
        for i in range(N):
            estimate[i], _, _ = least_square_method(
                xi[i, :], yi[i, :], ui[i, :], vi[i, :], 'inv')
        
        # compute R2 and confidence interval
        error3.append( np.nanmean( vort_drifters/true1 ) )
        error.append(stats.pearsonr(true2, estimate)[0]**2)
        error_l.append(stats.pearsonr(true1, estimate)[0]**2)
        error_l_ci.append(bootstrap_ci(true1, estimate, N))

    # save in dataframe
    df = pd.DataFrame(index=np.asarray(mlist))
    df['error1'] = np.asarray(error)
    df['error2'] = np.asarray(error_l)
    df['error3'] = np.asarray(error3)
    df['ci_low'] = np.asarray(error_l_ci)[:, 0]
    df['ci_high'] = np.asarray(error_l_ci)[:, 1]
    df['filter'] = filt

    return df

In [179]:
# %% MAIN
data_path = '../../data/'
zgrid_path = data_path+'psom/zgrid.out'
model_path = data_path+'psom/full_08325.cdf'

# dat = read_model_field(snakemake.input[0], snakemake.input[1])
dat = read_model_field(zgrid_path, model_path) 
dat = dat.rename({'xc':'x'})
dat = dat.rename({'yc':'y'})
dat = dat.set_coords({'x','y'})
dat = dat.transpose('x','y')

fu, fv, fzeta = filter_fields(dat)

In [180]:
N = 2500

length_bucket = []
number_bucket = []
aspect_bucket = []
for i, filt in enumerate([0, 10, 20]):
    print(filt)
    length_bucket.append(
        sensitivity_length(N, filt, fu=fu[i], fv=fv[i], fzeta=fzeta[i]))
    number_bucket.append(
        sensitivity_number(N, filt, fu=fu[i], fv=fv[i], fzeta=fzeta[i]))
    aspect_bucket.append(
        sensitivity_aspect(N, filt, fu=fu[i], fv=fv[i], fzeta=fzeta[i]))

0


ValueError: Length of values does not match length of index

In [ ]:
pd.concat(length_bucket).reset_index().to_feather(data_path+'psom/sensitivity_length.feather')
pd.concat(number_bucket).reset_index().to_feather(data_path+'psom/sensitivity_number.feather')
pd.concat(aspect_bucket).reset_index().to_feather(data_path+'psom/sensitivity_aspect.feather')